In [5]:
import torch
import pickle
import numpy as np
import torch
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from plots import plot_data, plot_metrics, plot_spatial_error_distributon
from utils import CustomDataset, parse_args, generate_cos_wave
from models import MLP

In [23]:
DATA_DIM = 10
torch.manual_seed(0)
np.random.seed(0)
N = 20
freq = 1.0

bs = 1
n = 128
lr = 0.001
epochs = 10000
device = torch.device("cuda")

# LOAD DATA FROM PICKLES

In [18]:
x = pickle.load(open('pickles/data/{}_samples_{}_freq_train_data.pickle'.format(N, freq), 'rb'))
y = pickle.load(open('pickles/data/{}_samples_{}_freq_train_labels.pickle'.format(N, freq), 'rb'))
test_x = pickle.load(open('pickles/data/{}_freq_test_data.pickle'.format(freq), 'rb'))
test_y = pickle.load(open('pickles/data/{}_freq_test_labels.pickle'.format(freq), 'rb'))
rand_matrix = pickle.load(open('pickles/data/{}_{}_projection_matrix'.format(DATA_DIM, DATA_DIM), 'rb'))

# PROJECTION


In [19]:
# Append zeros to expand 2d to DATA_DIM
zeros_x = np.zeros((x.shape[0], DATA_DIM))
zeros_x[:, :x.shape[1]] = x
zeros_test_x = np.zeros((test_x.shape[0], DATA_DIM))
zeros_test_x[:, :test_x.shape[1]] = test_x

In [20]:
# get a random unitary matrix
q, r = np.linalg.qr(rand_matrix, mode='complete')
# project points on this matrix
q = q * 10  # change the variance of q
zeros_x = np.matmul(q, zeros_x.T).T
zeros_test_x = np.matmul(q, zeros_test_x.T).T
x = zeros_x
test_x = zeros_test_x

In [21]:
dataset = CustomDataset(x, y, device)
test_dataset = CustomDataset(test_x, test_y, device)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=False)

# MODEL

In [22]:
model = MLP(in_dim=DATA_DIM, n=n)
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()

# TRAIN-TEST

In [ ]:
mean_train_losses = []
mean_test_losses = []
avg_errors = []
for epoch in range(epochs):
    model.train()
    train_losses = []
    test_losses = []
    avg_error = torch.empty(len(dataloader), bs)
    for batch_idx, (x_batch, y_batch) in enumerate(dataloader):
        optimizer.zero_grad()
        y_pred = model(x_batch)
        y_pred = y_pred.view(-1)

        avg_error[batch_idx] = torch.abs(y_batch - y_pred) / torch.abs(y_batch)

        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (test_x_batch, test_y_batch) in enumerate(test_dataloader):
            test_y_pred = model(test_x_batch)
            test_y_pred = test_y_pred.view(-1)
            loss = loss_fn(test_y_pred, test_y_batch)

            test_losses.append(loss.item())

    mean_train_losses.append(np.mean(train_losses))
    mean_test_losses.append(np.mean(test_losses))

    avg_error = torch.mean(avg_error)
    avg_errors.append(avg_error)
    print('epoch : {}, train loss : {:.4f}, test loss : {:.4f}, Average Error : {}'.format(epoch + 1, np.mean(train_losses), np.mean(test_losses), avg_error))


# MORE METRICS

In [ ]:
y_preds = torch.empty(len(dataloader), args.bs)
for batch_idx, (x_batch, y_batch) in enumerate(dataloader):
    y_pred = model(x_batch)
    y_pred = y_pred.view(-1)
    y_preds[batch_idx] = y_pred

y_preds = y_preds.view(-1).to(device)
avg_norm_mse = torch.mean((dataset.y - y_preds)**2 / dataset.y**2)
print('Average Normalized MSE:{}'.format(avg_norm_mse))